In [1]:
import pandas as pd
import pickle
import tensorflow as tf

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Conv1D, Flatten, Dropout, MaxPooling1D

import sys
sys.path.append('../../..')
from smartFAQ.src.tokenPad import tokenization_padding

import os

print('Setup is ready')

Setup is ready


# Load Data
---
## Doc2vec model

In [2]:
#with open('../../data/d2v_pretrained_weights.pickle', 'rb') as handle:
#    d2v_pretrained_weights = pickle.load(handle)
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

In [ ]:
#pretrained_weights = d2v_pretrained_weights.wv.syn0
#vocab_size, embedding_size = pretrained_weights.shape

## Tokenization & Padding

In [3]:
MAX_NUM_WORDS = 20000


In [4]:
input_tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_train['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_train['answer'])

d2v_idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(d2v_idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 30369
Length of longest sentence in input: 1688


In [5]:
x_train_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)

print("encoder_input_sequences.shape:", x_train_pad.shape)
print("encoder_input_sequences[172]:", x_train_pad[70])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [    0     0     0 ... 18033   674     1]


In [6]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_val['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_val['answer'])

x_val_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)

print("encoder_input_sequences.shape:", x_val_pad.shape)
print("encoder_input_sequences[172]:", x_val_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [   0    0    0 ... 5837 5419 5838]


## Keras Model

In [7]:
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from gensim.models.phrases import Phrases, Phraser

In [ ]:
#sent = []
#for col in ['question', 'question_body', 'answer']:
#    words = [row.split() for row in X_train[col]]
#    sent = sent + words

In [ ]:
## Bigrams 

# phrases = Phrases(sent)
# bigram = Phraser(phrases)
# sentences = bigram[sentences]

In [ ]:
#documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]

In [ ]:
#d2v_model = Doc2Vec(min_count=50,
 #                    window=2,
  #                   size=300, 
   #                  alpha=0.025, 
    #                 min_alpha=0.025)

In [ ]:
# sentences

In [8]:
with open('../../data/d2v_pretrained_weights.pickle', 'rb') as handle:
    d2v_pretrained_weights = pickle.load(handle)

In [9]:
pretrained_weights = d2v_pretrained_weights.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [10]:
d2v_pretrained_weights.most_similar(['import'])

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('env_python', 0.34262266755104065),
 ('import_sys', 0.3281152844429016),
 ('import_glob', 0.31906190514564514),
 ('import_threading', 0.30781054496765137),
 ('import_os', 0.2753518223762512),
 ('selenium_import', 0.267800509929657),
 ('collections_import', 0.2605658769607544),
 ('import_matplotlib', 0.2559884488582611),
 ('bs_import', 0.2548660337924957),
 ('tkinter_import', 0.2535885274410248)]

# Build

In [11]:
def build_model(vocab_size,embedding_size,pretrained_weights):
    
           
    model = tf.keras.Sequential()
    
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_size, 
                        weights=[pretrained_weights],
                        input_length=max_input_len
                       ))

    model.add(LSTM(units=embedding_size))
    model.add(Dense(units=vocab_size))
    model.add(Activation('softmax'))
    model.add(Dense(1))

    return model

In [12]:
def compile_model(model):
    mse = tf.keras.losses.MeanSquaredError()
    model.compile(loss=mse,
                  optimizer='adam',
                  metrics=['MeanSquaredError']
                  )
    return  model

In [13]:
def fit_model( x_train, y_train, x_val, y_val, model, batch_size,  epochs = 5):
    
    print('Train...')
    os.makedirs("./logs",exist_ok=True)
    tensorboard = TensorBoard(log_dir=os.path.join('./logs'), histogram_freq=0,
                                  write_graph=True, write_images=False,profile_batch = 100000000)

    callbacks = [ tensorboard]

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_val, y_val),
              callbacks= callbacks)
    return model

In [14]:
with tf.device("/CPU:0"):
    model = build_model(vocab_size, embedding_size, pretrained_weights)
    model.summary()
    model = compile_model(model)
    model = fit_model(x_train_pad, y_train, x_val_pad, y_val, model, batch_size=200, epochs=10)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1688, 300)         1131600   
_________________________________________________________________
lstm (LSTM)                  (None, 300)               721200    
_________________________________________________________________
dense (Dense)                (None, 3772)              1135372   
_________________________________________________________________
activation (Activation)      (None, 3772)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3773      
Total params: 2,991,945
Trainable params: 2,991,945
Non-trainable params: 0
_________________________________________________________________
Train...
Epoch 1/10


InvalidArgumentError:  indices[195,1651] = 4138 is not in [0, 3772)
	 [[node sequential/embedding/embedding_lookup (defined at <ipython-input-13-23f7722d067e>:14) ]] [Op:__inference_train_function_2783]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/1811 (defined at C:\Users\achenu\Anaconda3\envs\FAQ\lib\contextlib.py:112)

Function call stack:
train_function


In [15]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('./saved_model/LSTM_model_d2v')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./saved_model/LSTM_model_d2v\assets


In [16]:
(loss, acc) = model.evaluate(x_val_pad, y_val)

InvalidArgumentError:  indices[31,1651] = 10483 is not in [0, 3772)
	 [[node sequential/embedding/embedding_lookup (defined at <ipython-input-16-2bc3015b0f35>:1) ]] [Op:__inference_test_function_9965]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/9461 (defined at C:\Users\achenu\Anaconda3\envs\FAQ\lib\contextlib.py:112)

Function call stack:
test_function


In [ ]:
# © Laëtitia CONSTANTIN & Axel CHENU 2021

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e80043e2-6875-4b65-a196-a0ffb97a1282' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>